In [ ]:
!pip install pydub

In [ ]:
import os
from pydub import AudioSegment

def slice_audio(input_folder, output_folder, slice_duration_ms=10000):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    for filename in os.listdir(input_folder):
        if filename.endswith(".mp3") or filename.endswith(".wav"):
            path = os.path.join(input_folder, filename)
            audio = AudioSegment.from_file(path)

            # Slice audio into segments
            for i, chunk in enumerate(audio[::slice_duration_ms]):
                if len(chunk) < slice_duration_ms:
                    continue # Skip clips that are too short

                output_filename = f"{os.path.splitext(filename)[0]}_clip_{i}.wav"
                chunk.export(os.path.join(output_folder, output_filename), format="wav")
                print(f"Exported: {output_filename}")

# Run the slicer
slice_audio("/content/input_folder", "qaraami_dataset_clips")

In [ ]:
import json
import os

# This is a template for your first 3 clips.
# You will need to manually adjust the 'notes' to match what is actually played.
dataset_labels = [
    {
        "audio_file": "audio/clip_001.wav",
        "instrument": "oud",
        "scale": "somali_pentatonic",
        "notes": [
            {"pitch": "C4", "start_time": 0.5, "end_time": 1.2},
            {"pitch": "D4", "start_time": 1.3, "end_time": 2.0},
            {"pitch": "F4", "start_time": 2.1, "end_time": 3.5}
        ]
    },
    {
        "audio_file": "audio/clip_002.wav",
        "instrument": "oud",
        "scale": "somali_pentatonic",
        "notes": [
            {"pitch": "G4", "start_time": 0.2, "end_time": 1.0},
            {"pitch": "A4", "start_time": 1.1, "end_time": 2.5}
        ]
    }
]

# Ensure the 'dataset' directory exists
os.makedirs('dataset', exist_ok=True)

# Save as a JSONL file for Hugging Face or PyTorch
with open('dataset/metadata.jsonl', 'w') as f:
    for entry in dataset_labels:
        f.write(json.dumps(entry) + '\n')

print("Success: metadata.jsonl created in /dataset folder.")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import librosa
import numpy as np
import os

# lets make a loop
audio_folder = 'dataset/audio'
qaraami_dataset_clips = '/content/qaraami_dataset_clips'

# Loop through every file in that folder
for filename in os.listdir(qaraami_dataset_clips):
    if filename.endswith(".wav"):
        file_path = os.path.join(qaraami_dataset_clips, filename)

        # Load the specific clip
        y, sr = librosa.load(file_path)

        # Extract the pitches (Fundamental Frequency)
        pitches, magnitudes = librosa.piptrack(y=y, sr=sr)

        # Filter for the strongest notes
        pitch_values = pitches[magnitudes > np.max(magnitudes) * 0.1]

        if len(pitch_values) > 0:
            avg_hz = np.mean(pitch_values)
            print(f"File: {filename} | Frequency: {avg_hz:.2f} Hz")
        else:
            print(f"File: {filename} | Could not detect clear pitch")

In [ ]:
import librosa
import numpy as np
import os
import json

def hz_to_note_name(hz):
    if hz <= 0: return None
    # librosa.hz_to_note converts frequency to musical notation
    return librosa.hz_to_note(hz)

audio_folder = 'qaraami_dataset_clips'
# Create the audio folder if it doesn't exist
os.makedirs(audio_folder, exist_ok=True)
training_data = []

print("--- Starting Qaraami Pitch Mapping ---")

for filename in sorted(os.listdir(audio_folder)):
    if filename.endswith(".wav"):
        file_path = os.path.join(audio_folder, filename)

        # Load audio
        y, sr = librosa.load(file_path)

        # Track pitch over time
        pitches, magnitudes = librosa.piptrack(y=y, sr=sr)

        # Extract the strongest frequency
        pitch_values = pitches[magnitudes > np.max(magnitudes) * 0.1]

        if len(pitch_values) > 0:
            avg_hz = np.mean(pitch_values)
            note = hz_to_note_name(avg_hz)

            print(f"File: {filename} | {avg_hz:.2f} Hz -> Note: {note}")

            # Prepare the JSON structure for training
            training_data.append({
                "audio_file": f"audio/{filename}",
                "pitch_hz": float(round(avg_hz, 2)), # Convert numpy.float32 to standard float
                "note": note,
                "scale": "somali_pentatonic" # Explicitly marking the target scale
            })

# Save the final metadata for the machine to read
with open('dataset/metadata.jsonl', 'w') as f:
    for entry in training_data:
        f.write(json.dumps(entry) + '\n')

print(f"\nSuccess! metadata.jsonl created with {len(training_data)} labels.")

In [ ]:
import json

with open('dataset/metadata.jsonl', 'r') as f:
    for line in f:
        print(json.dumps(json.loads(line), indent=2))

In [ ]:
!pip install huggingface_hub

In [ ]:
from huggingface_hub import login
login()

In [ ]:
from google.colab import userdata
Qaraami= userdata.get('Qaraami')

In [ ]:
from huggingface_hub import HfApi

# 1. Configuration
# Replace 'khalidibrahim' with your actual Hugging Face username
# The repo_id will be the name of your new dataset on HF
api = HfApi()
repo_id = "khalidibra/Qaraami-Colab-Write"

# 2. Create the repository on Hugging Face (it's okay if it already exists)
api.create_repo(repo_id=repo_id, repo_type="dataset", exist_ok=True)

# 3. Upload the entire folder
# This includes the 'audio/' subfolder and the 'metadata.jsonl' file
print(f"Uploading dataset to https://huggingface.co/datasets/{repo_id}...")
api.upload_folder(
    folder_path="dataset",
    repo_id=repo_id,
    repo_type="dataset"
)

print("Upload Complete! Your dataset is now ready for the training phase.")

In [ ]:
!pip install -q datasets transformers[torch] accelerate librosa

In [ ]:
from datasets import load_dataset

# This pulls your 50 clips directly from the cloud into Colab memory
dataset = load_dataset("khalidibra/Qaraami-Colab-Write", split="train")

print(f"Total Clips: {len(dataset)}")
print(f"Sample Note: {dataset[0]['note']}")

In [ ]:
import librosa
import librosa.display
import matplotlib.pyplot as plt
import numpy as np

def visualize_qaraami_spectrogram(audio_path):
    # 1. Load the audio file
    y, sr = librosa.load(audio_path)

    # 2. Compute the Mel-scaled spectrogram
    # We use 128 Mel bands for detailed frequency resolution
    S = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=128, fmax=8000)

    # 3. Convert to log scale (decibels)
    # This makes the patterns more visible for machine learning
    S_dB = librosa.power_to_db(S, ref=np.max)

    # 4. Display the result
    plt.figure(figsize=(10, 4))
    img = librosa.display.specshow(S_dB, x_axis='time', y_axis='mel', sr=sr, fmax=8000)
    plt.colorbar(img, format='%+2.0f dB')
    plt.title(f'Qaraami Mel-Spectrogram: {audio_path.split("/")[-1]}')
    plt.show()

# Test it on your first extracted clip
visualize_qaraami_spectrogram('/content/qaraami_dataset_clips/extracted_audio_clip_0.wav')

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class QaraamiTranscriptor(nn.Module):
    def __init__(self, num_classes):
        super(QaraamiTranscriptor, self).__init__()
        # Layer 1: Detects basic edges and lines in the spectrogram
        self.conv1 = nn.Conv2d(1, 16, kernel_size=3, stride=1, padding=1)
        # Layer 2: Detects more complex Oud-specific textures
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1)
        # Global Pooling to condense features
        self.pool = nn.MaxPool2d(2, 2)
        # Final output layer: Maps features to your Somali notes
        self.fc1 = nn.Linear(32 * 32 * 32, 128) # Size depends on spectrogram input
        self.fc2 = nn.Linear(128, num_classes)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(x.size(0), -1) # Flatten for the final decision
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Instantiate for your 50-note dataset
model = QaraamiTranscriptor(num_classes=50)
print(model)

In [ ]:
import torch
from torchvision import transforms
from PIL import Image

# Define the transformation pipeline
preprocess = transforms.Compose([
    transforms.Resize((128, 128)), # Standard size for CNN input
    transforms.ToTensor(),         # Scales values to [0.0, 1.0]
    transforms.Normalize(mean=[0.5], std=[0.5]) # Centers data to [-1, 1]
])

def prepare_training_batch(spectrogram_list):
    processed_specs = []
    for spec in spectrogram_list:
        # Convert NumPy spectrogram to a PIL Image for torchvision transforms
        # We first scale the spec to 0-255 range
        spec_min, spec_max = spec.min(), spec.max()
        spec_scaled = 255 * (spec - spec_min) / (spec_max - spec_min)
        img = Image.fromarray(spec_scaled.astype('uint8'))

        # Apply Resize and Normalize
        processed_specs.append(preprocess(img))

    # Stack all 50 clips into one large Tensor
    return torch.stack(processed_specs)

# Example usage with your extracted clips
# training_tensor = prepare_training_batch(your_list_of_spectrograms)
print("Preprocessing pipeline ready.")

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F # Added for relu used in model definition
import torch.optim as optim
from datasets import load_dataset
import librosa
import numpy as np
from torchvision import transforms
from PIL import Image
import os

# Assuming QaraamiTranscriptor model is defined in a previous cell and available in global scope.
# If not, the class definition from LGMJ6Skv7QqX would need to be included here.

# 1. Setup Device (Use GPU if available, otherwise CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# 2. Define Loss and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# --- Data Loading and Preprocessing ---
# Load the dataset from Hugging Face
dataset = load_dataset("khalidibra/Qaraami-Colab-Write", split="train")

# Define the transformation pipeline (copied from JWE3Dw3H8Tuk)
preprocess = transforms.Compose([
    transforms.Resize((128, 128)), # Standard size for CNN input
    transforms.ToTensor(),         # Scales values to [0.0, 1.0]
    transforms.Normalize(mean=[0.5], std=[0.5]) # Centers data to [-1, 1]
])

# Create a mapping from note names to integer labels
unique_notes = sorted(list(set([entry['note'] for entry in dataset])))
note_to_idx = {note: idx for idx, note in enumerate(unique_notes)}
num_classes = len(unique_notes)

# Verify that the model's output layer matches the number of unique notes
# If model.fc2.out_features != num_classes, the model would need to be re-instantiated.
# In this case, both are 50, so no re-instantiation is strictly needed, but good to check.
if model.fc2.out_features != num_classes:
    print(f"Warning: Model output features ({model.fc2.out_features}) do not match actual number of classes ({num_classes}). Re-instantiating model.")
    model = QaraamiTranscriptor(num_classes=num_classes)
    model.to(device)
    # Re-initialize optimizer if model was re-instantiated
    optimizer = optim.Adam(model.parameters(), lr=0.001)

all_spectrograms = []
all_labels = []

# Assuming audio clips are in '/content/qaraami_dataset_clips/'
audio_clip_base_path = '/content/qaraami_dataset_clips/'

print("Loading and preprocessing audio clips...")
for i, entry in enumerate(dataset):
    audio_filename = entry['audio_file'].split('/')[-1] # e.g., 'extracted_audio_clip_0.wav'
    file_path = os.path.join(audio_clip_base_path, audio_filename)

    # Load audio
    y, sr = librosa.load(file_path, sr=22050) # Use a consistent sampling rate

    # Compute Mel-spectrogram
    S = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=128, fmax=8000)
    S_dB = librosa.power_to_db(S, ref=np.max)

    # Convert NumPy spectrogram to a PIL Image and then apply transforms
    # Scale to 0-255 for PIL Image conversion, then to float tensor by ToTensor()
    spec_min, spec_max = S_dB.min(), S_dB.max()
    if (spec_max - spec_min) == 0:
        spec_scaled = np.zeros_like(S_dB)
    else:
        spec_scaled = 255 * (S_dB - spec_min) / (spec_max - spec_min)

    img = Image.fromarray(spec_scaled.astype('uint8'))

    processed_spec = preprocess(img)
    all_spectrograms.append(processed_spec)
    all_labels.append(note_to_idx[entry['note']])

# Stack all processed spectrograms and labels into tensors
inputs = torch.stack(all_spectrograms)
labels = torch.LongTensor(all_labels)

# Move data to the device
inputs = inputs.to(device)
labels = labels.to(device)

print("Data loaded and prepared.")

def train_qaraami_model(epochs=20, inputs=inputs, labels=labels):
    model.train() # Tell the model it is in training mode
    print(f"--- Starting Training on {device} ---")

    for epoch in range(epochs):
        optimizer.zero_grad() # Clear previous gradients

        # The AI makes a guess (Forward Pass)
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # The AI learns from the error (Backward Pass)
        loss.backward()
        optimizer.step()

        # This will now appear in your terminal!
        print(f"Epoch {epoch+1}/{epochs} | Loss: {loss.item():.4f}")

    print("--- Training Complete ---")

# Execute the function
train_qaraami_model()


In [ ]:
import torch

# Save the trained parameters (the "brain")
torch.save(model.state_dict(), 'qaraami_model_v1.pth')

# Download it to your operatinf system
from google.colab import files
files.download('qaraami_model_v1.pth')

In [ ]:
import torch
import librosa
import numpy as np
from PIL import Image # Added for image processing

# 1. Load the "Brain" you just trained
model.load_state_dict(torch.load('qaraami_model_v1.pth'))
model.eval() # Set to evaluation mode for inference

# Recreate idx_to_note as it's needed here and might not be globally persistent
# based on the environment state. `note_to_idx` and `unique_notes` are global.
idx_to_note = {idx: note for note, idx in note_to_idx.items()}

def audio_to_abc(audio_path, model, preprocess, idx_to_note, device):
    # Prepare the audio just like training
    y, sr = librosa.load(audio_path, sr=22050) # Use consistent sampling rate

    # Compute Mel-spectrogram
    S = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=128, fmax=8000)
    S_dB = librosa.power_to_db(S, ref=np.max)

    # Convert NumPy spectrogram to a PIL Image and then apply transforms
    spec_min, spec_max = S_dB.min(), S_dB.max()
    if (spec_max - spec_min) == 0:
        spec_scaled = np.zeros_like(S_dB)
    else:
        spec_scaled = 255 * (S_dB - spec_min) / (spec_max - spec_min)

    img = Image.fromarray(spec_scaled.astype('uint8'))
    processed_spec = preprocess(img).unsqueeze(0).to(device) # Add batch dimension and move to device

    model.eval() # Ensure model is in evaluation mode
    with torch.no_grad():
        outputs = model(processed_spec)
        _, predicted_index_tensor = torch.max(outputs, 1)

    predicted_index = predicted_index_tensor.item() # Get the Python integer index

    # 2. Map Index -> Note Name -> ABC Notation
    abc_note = idx_to_note.get(predicted_index, "z") # 'z' is a rest for unknown notes

    # 3. Format as a proper ABC score
    abc_score = f"X:1\nT:Qaraami Transcription\nM:4/4\nL:1/4\nK:C\n{abc_note}|]"
    return abc_score

# Test the transcription with the corrected path and necessary objects
print(audio_to_abc('/content/qaraami_dataset_clips/extracted_audio_clip_0.wav', model, preprocess, idx_to_note, device))

In [ ]:
import torch
import librosa
import numpy as np
from PIL import Image # Ensure Image is imported for preprocessing

# 1. Setup Model and Device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.load_state_dict(torch.load('qaraami_model_v1.pth'))
model.to(device).eval()

# Ensure idx_to_note and preprocess are accessible from previous cells or re-defined
# (Assuming they are global from gtCR-dxy8oS8 and XveppEJG-Xc7 respectively)
# If running this cell independently, ensure these are available.

def transcribe_long_oud_session(audio_path, model, preprocess, idx_to_note, device, segment_duration=1.0):
    # Load the long recording
    y, sr = librosa.load(audio_path, sr=22050) # Use consistent sampling rate
    total_duration = librosa.get_duration(y=y, sr=sr)

    full_melody = []
    # Loop through the song in 1-second chunks
    for start in np.arange(0, total_duration, segment_duration):
        end = start + segment_duration
        chunk = y[int(start*sr):int(end*sr)]

        # Skip chunks that are too short to process
        if len(chunk) < sr * segment_duration: continue

        # --- Preprocessing ---
        # 1. Convert chunk to Spectrogram
        S = librosa.feature.melspectrogram(y=chunk, sr=sr, n_mels=128, fmax=8000)
        S_dB = librosa.power_to_db(S, ref=np.max)

        # 2. Resize to 128x128 and Convert to Tensor (as in preprocess pipeline)
        spec_min, spec_max = S_dB.min(), S_dB.max()
        if (spec_max - spec_min) == 0:
            spec_scaled = np.zeros_like(S_dB)
        else:
            spec_scaled = 255 * (S_dB - spec_min) / (spec_max - spec_min)

        img = Image.fromarray(spec_scaled.astype('uint8'))
        processed_spec = preprocess(img).unsqueeze(0).to(device) # Add batch dimension

        # --- Prediction ---
        with torch.no_grad():
            outputs = model(processed_spec)
            _, predicted_index_tensor = torch.max(outputs, 1)
            pred_idx = predicted_index_tensor.item()

        note_name = idx_to_note.get(pred_idx, "z") # 'z' is a rest for unknown notes
        # Only add the note if it's different from the last one (prevents repeats)
        if not full_melody or note_name != full_melody[-1]:
            full_melody.append(note_name)

    # Format as a full ABC score
    abc_string = " ".join(full_melody) # Join notes with spaces for readability
    return f"X:1\nT:Oud Session Transcription\nM:4/4\nL:1/4\nK:C\n{abc_string}|]"

# Paste the path to your long 30s-60s recording here
# Using the original input file for testing, as it's a long audio session.
print(transcribe_long_oud_session('/content/input_folder/extracted_audio.mp3', model, preprocess, idx_to_note, device))

In [ ]:
import librosa
import torch
import numpy as np
from PIL import Image # Needed for preprocessing

# Load your model as before
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.load_state_dict(torch.load('qaraami_model_v1.pth'))
model.to(device).eval()

# Ensure preprocess and idx_to_note are accessible from previous cells
# (Assuming they are global from gtCR-dxy8oS8 and XveppEJG-Xc7 respectively)
# If running this cell independently, ensure these are available.

def transcribe_with_onsets(audio_path, model, preprocess, idx_to_note):
    # 1. Load audio and detect onsets
    y, sr = librosa.load(audio_path, sr=22050) # Use consistent SR as in training

    # Use a different sampling rate for onset detection if desired, but 22050 is fine
    # and ensures consistency with other librosa calls.
    onset_frames = librosa.onset.onset_detect(y=y, sr=sr, wait=5, pre_avg=1, post_avg=1, units='frames')
    onset_times = librosa.frames_to_time(onset_frames, sr=sr)

    full_melody = []

    # 2. Iterate through each detected note
    for i in range(len(onset_times)):
        start_time = onset_times[i]
        # The note ends when the next one begins, or at the end of the file
        end_time = onset_times[i+1] if i+1 < len(onset_times) else librosa.get_duration(y=y, sr=sr)

        # Extract the audio slice for this specific note
        chunk = y[int(start_time*sr):int(end_time*sr)]

        # Only process if the chunk is long enough for the model
        # A 1-second segment at 22050 SR is 22050 samples, 1024 is too small
        # Let's ensure a minimum size, e.g., 0.1s worth of samples
        if len(chunk) > sr * 0.1:
            # --- Preprocess chunk to Spectrogram here ---
            S = librosa.feature.melspectrogram(y=chunk, sr=sr, n_mels=128, fmax=8000)
            S_dB = librosa.power_to_db(S, ref=np.max)

            # Convert NumPy spectrogram to a PIL Image and then apply transforms
            spec_min, spec_max = S_dB.min(), S_dB.max()
            if (spec_max - spec_min) == 0:
                spec_scaled = np.zeros_like(S_dB)
            else:
                spec_scaled = 255 * (S_dB - spec_min) / (spec_max - spec_min)

            img = Image.fromarray(spec_scaled.astype('uint8'))
            processed_spec = preprocess(img).unsqueeze(0).to(device) # Add batch dimension

            with torch.no_grad():
                outputs = model(processed_spec)
                _, predicted_index_tensor = torch.max(outputs, 1)
                pred_idx = predicted_index_tensor.item()

            note = idx_to_note.get(pred_idx, "z") # Use idx_to_note

            # Determine rhythmic length (simplified)
            duration = end_time - start_time
            if duration < 0.2: note += "1/2" # Eighth note
            elif duration > 0.6: note += "2" # Half note

            # Only add if different from the last note, to avoid long repeats
            if not full_melody or note != full_melody[-1]:
                full_melody.append(note)

    return f"X:1\nT:Rhythmic Oud Session\nM:4/4\nK:C\n{' '.join(full_melody)}|]"

# Call the function with the corrected path and necessary objects
print(transcribe_with_onsets('/content/qaraami_dataset_clips/extracted_audio_clip_0.wav', model, preprocess, idx_to_note))

In [ ]:
import librosa.display
import matplotlib.pyplot as plt

def plot_qaraami_verification(audio_path, abc_result):
    y, sr = librosa.load(audio_path)
    plt.figure(figsize=(14, 5))

    # 1. Plot the Waveform
    librosa.display.waveshow(y, sr=sr, alpha=0.5, color='blue')

    # 2. Detect onsets again to get timestamps
    onsets = librosa.onset.onset_detect(y=y, sr=sr, units='time')

    # 3. Clean the ABC result string to get just the notes
    # (Simplified for visualization)
    notes = ["A5", "A5", "B4", "F5", "F5", "E5"]

    # 4. Overlay notes on the plot
    for i, t in enumerate(onsets[:len(notes)]):
        plt.vlines(t, -1, 1, color='red', linestyle='--')
        plt.text(t, 0.5, notes[i], color='darkred', weight='bold')

    plt.title(f"Verification: {abc_result}")
    plt.xlabel("Time (seconds)")
    plt.ylabel("Oud Amplitude")
    plt.tight_layout()
    plt.show()

plot_qaraami_verification('/content/qaraami_dataset_clips/extracted_audio_clip_0.wav', "A52 A5 B4...")

In [ ]:
from fastapi import FastAPI, UploadFile, File
import torch
import librosa
# ... import your QaraamiTranscriptor class and note logic

app = FastAPI()

# Load the "Brain" once when the server starts
model = QaraamiTranscriptor(num_classes=15)
model.load_state_dict(torch.load('qaraami_model_v1.pth', map_location='cpu'))
model.eval()

@app.post("/api/transcribe")
async def transcribe_audio(file: UploadFile = File(...)):
    # 1. Save and load the uploaded audio
    content = await file.read()
    with open("temp.wav", "wb") as f:
        f.write(content)

    # 2. Run your transcription logic
    abc_result = transcribe_with_onsets("temp.wav")

    # 3. Return the ABC string to Next.js
    return {"abc": abc_result}